### Create a Virtual Environment
Create a folder in the current directory which will contain the Python executable files, and a copy of the pip library which you can use to install other packages. 

	$ virtualenv env

Activate Virtual Env.

	$ source env/bin/activate
	
Install Pandas OR maybe add REQUIREMENTS.txt
    
    $ pip install pandas
	